In [1]:
import arviz
import cmdstanpy
from IPython.display import display
from matplotlib import pyplot as plt
from numbers import Number
import numpy as np
import os
import pandas as pd
from pandas.io.json import json_normalize
import toml
import reaction_network
plt.style.use('sparse.mplstyle')

# Modelling networks of metabolic reactions with Stan

Many useful substances can be produced using chemical reactions that happen inside living organisms like _E. Coli,_ yeast or Chinese hamster ovaries. Biological production is appealing because it can be done in relatively mild conditions. On the other hand, biological production depends on complex networks of interacting enzyme-catalysed chemical reactions which can be difficult to understand, manipulate and optimise.

This notebook describes how to analyse metabolic networks by integrating different sources of information - structural knowledge, pre-experimental information about parameters and experiment results - using Bayesian inference.

## Wait, what is an enzyme-catalysed reaction?
An enzyme is a 'catalyst' protein that speeds up a chemical reaction by binding the reactants to an 'active site' where the conditions for the reaction are favourable. This visualisation gives an idea of how this process is thought to work: 

![Enzyme Action and the Hydrolysis of Sucrose [HD Animation]](https://i.makeagif.com/media/9-16-2015/SIif27.gif)

## Structural information 

The topological structure of most industrially relevant metabolic networks has been mapped. It is usually known which substances go in and out of each reaction the enzyme that catalyses it. Often there is even more structural information, such as what shape the enzyme has, which substances can bind to its active site.

This information is available online, for example in the [BIGG database](http://bigg.ucsd.edu/). 

Here is a reduced map of yeast central metabolism, with nodes representing metabolites and edges representing reactions: 

<img src="img/yeast_central_metabolism.png" alt="Drawing" style="width: 600px;"/>

Finer-grained structural information can be used to deduce the mechanism by which each enzyme operates. A mechanism, in turn, implies a way of parameterising the rate at which an enzyme turns substrates into products. For example, the following equation describes how the rate $v$ of a 'uni-uni' mechanism (See [Cleland (1963)](https://doi.org/10.1016/0926-6569%2863%2990211-6) for more about mechanisms in general and derivations of particular mechanisms used in this analysis) with no activators or inhibitors depends on the current concentration of its substrate $A$ and product $P$, as well as parameters $K_{eq}$, $K_{a}$, $V1$ and $V2$:

$$
\begin{equation}
v = \frac{V1\cdot{V2}\cdot{(A-P/K_{eq})}}{K_{a}\cdot{V2} + V2\cdot{A} + V1\cdot{P/K_{eq}}}
\end{equation}
$$

The main output of this analysis will be inference about the values of this kind of parameter.


## Pre-experimental information
Most parameters are difficult to assess because the conditions inside a cell are difficult to replicate _in vitro_. See [Saa and Keld Nielsen (2017)](https://doi.org/10.1016/j.biotechadv.2017.09.005) for more about top-down vs bottom-up parameter estimation. Nonetheless, a lot of information is available.

For kinetic parameters it is possible to use the 'Respectful' modelling paradigm (see [Tsigkinopoulou, Baker, and Breitling (2017)](https://doi.org/10.1016/j.tibtech.2016.12.008)) to aggregate the results of findings about kinetic parameters published in the [BRENDA enzyme database](https://www.brenda-enzymes.org/), as well as assessments of thermodynamic parameters from the [equilibrator database](http://equilibrator.weizmann.ac.il/). This analysis produces an independent normal or log-normal distribution representing the pre-experimental information available about each parameter.

## Experimental information

Modern experimental techniques measure the following quantities:

- Enzyme concentrations (_proteomics_)
- Metabolite concentrations (_metabolomics_)
- Rates of reactions, aka fluxes (_fluxomics_)

Each kind of measurement is noisy.

## Modelling strategy

We are proposing to use Bayesian inference to combine all the available sources of information about a metabolic network.

Pre-experimental information supplies informative prior probability distributions about the parameters that describe each reaction.

We treat information from metabolomics and fluxomics data using a regression model where recorded values of measured quantities are draws from distributions centered on the true values as follows, with indexes $e$ standing for experiments, $f$ for fluxes and $m$ for metabolites:

$$
measured\, flux_{f,e} \sim Normal(true\, flux_{f,e}, \sigma_{f,e}) \\
measured\, metabolite\, concentration_{m,e} \sim Lognormal(\log(true\, metabolite\, concentration_{m,e}), \sigma_{m,e})
$$

Structural information can be used to connect the other two sources of information by mapping parameter configurations to true metabolite concentrations and fluxes. This is done by expressing the evolution of each metabolite's concentration as a system of ordinary differential equations, with the rate of change of each metabolite determined by the rates of the reactions that produce and consume it. We assume that, at the time of measurement, the concentration of the network's internal metabolites is constant, or in other words that the system has reached a steady state. In general, a vector $\mathbf{v}$ of steady state fluxes must satisfy the following condition

$$
S\cdot\mathbf{v} = \mathbf{0}
$$

Where $S$ is a known 'stoichiometric matrix' indicating which reactions produce and consume which metabolites and $\mathbf{0}$ is a vector of as many zeros as the network has internal metabolites. Given a parameter configuration, a steady state metabolite concentration can be found by solving the resulting system of algebraic equations. 

We solve these equation systems by integrating the implied ODEs over a period of time for which it is safe to assume that the system has reached steady state.

This approach is essentially the same as that set out by [Saa and Keld Nielsen (2016)](https://doi.org/10.1038/srep29635), but uses MCMC via Stan rather than rejection sampling to make inferences. Thanks to Stan's intuitive model language, this approach and attendant assumptions can be represented in a more or less self-documenting form.


## Example - a simple linear system

The rest of the notebook applies this modelling strategy to investigate how accurately it is possible to find out the value of the kinetic parameters and steady state metabolite concentrations of the simple metabolic network depicted below, given a known experimental setup and prior information.

<img src="img/linear.png" alt="Drawing" style="width: 800px;"/>

This network has two internal metabolites, M1 and M2, and three reactions R1, R2 and R3. Metabolites M1 and M2 are measured, and reaction R2 is inhibited by M1. The test tube photos indicate quantities that were measured.

To gauge how accurately the available information identifies the quantities of interest, the notebook uses Stan to generate simulated experimental data given 'true' parameter values, then uses the simulated data as model input. If the model is unable to recover the true value of a parameter, this is may be a sign that the simulated experiment shouldn't be carried out in real life without more diverse or accurate measurements or better pre-experimental information.

## Model input
We are developing an input data format for succinctly expressing the information required to model a metabolic network. Here is what the linear system looks like in this format:

In [2]:
with open('linear.toml', 'r') as f:
    print(f.read())

[constants]
gas_constant = 8.314e-3 # units kJ/mol.K

[[experiment]]
label = 'condition_1'
concentration_r1 = 0.3
concentration_r2 = 1
concentration_r3 = 1
unbalanced_metabolite_priors = [
  { label = 'M1_ex', loc = 1, scale = 0.05, true_value = 2.2 },
  { label = 'M2_ex', loc = 0, scale = 0.05, true_value = 1.3 },
]
measurements = [
  { label = 'M1', value = 0.8, scale = 0.1, type = 'metabolite' },
  { label = 'M2', value = 1.5, scale = 0.1, type = 'metabolite' },
  { label = 'r3', value = 0.29, scale = 0.1, type = 'flux' }
]

[[experiment]]
label = 'condition_2'
concentration_r1 = 0.2
concentration_r2 = 1
concentration_r3 = 1
unbalanced_metabolite_priors = [
  { label = 'M1_ex', loc = 1, scale = 0.05, true_value = 2.9 },
  { label = 'M2_ex', loc = 0, scale = 0.05, true_value = 0.7 },
]
measurements = [
  { label = 'M1', value = 0.7, scale = 0.1, type = 'metabolite' },
  { label = 'M2', value = 1.4, scale = 0.1, type = 'metabolite' },
  { label = 'r3', value = 0.21, scale = 0.1, type 

The `reaction` entries in this file jointly specify the network's stoichiometry, reaction mechanisms, priors and allosteric interactions, while the `experiment` entries describe how the system has been measured, including experimental conditions, measured quantities and accuracies of each measurement. A value for each measurement can also be specified, but here we will simulate these using the specified parameter `true_value`s. It is assumed that all experiments were carried out at the same temperature.

### Allosteric regulation

Allosteric effects occur when an enzyme molecule's shape changes due to substances binding to it, making the enzyme a more or less effective catalyst. Allosteric effects play an important role in regulating metabolic reactions. Following [Popova and Selkov (1975)](https://doi.org/10.1016/0014-5793(75)80034-2), we model allosteric regulation through a function that expresses the overall effect of allosteric factors on the rate of a reaction as a multiplicative factor. Here is what the function looks like in Stan code:

In [3]:
with open('stan_code/allostery.stan', 'r') as f:
    print(f.read())

real get_free_enzyme_ratio_uniuni(real A, real P, real V1, real V2, real Ka, real Keq){
  real Kp = Keq * V2 * Ka / V1;
  return 1 / (1 + A/Ka + P/Kp);
}
real get_regulatory_effect(real[] activator_concentration,
                           real[] inhibitor_concentration,
                           real free_enzyme_ratio,
                           real[] dissociation_constant_r,
                           real[] dissociation_constant_t,
                           real transfer_constant){
  real Q_num = size(inhibitor_concentration) == 0 ? 1 :
    1 + sum(to_vector(inhibitor_concentration) ./ to_vector(dissociation_constant_t));
  real Q_denom = size(activator_concentration) == 0 ? 1
    : 1 + sum(to_vector(activator_concentration) ./ to_vector(dissociation_constant_r));
  real Q = transfer_constant * free_enzyme_ratio * Q_num / Q_denom;
  return inv(1 + Q); 
}
    



### Rate equations

The following Stan functions describe the catalytic effects of each reaction mechanism that appears in the network we want to analyse

In [4]:
with open('stan_code/rate_equations.stan', 'r') as f:
    print(f.read())

real uniuni(real A, real P, real V1, real V2, real Ka, real Keq){
  real num = V1*V2*(A-P/Keq);
  real denom = Ka*V2 + V2*A + V1*P/Keq;
  return num / denom;
}



### Inference

The model for making inferences is shown below. The `functions` block uses the library functions described above to define an ode function called `steady_state_equation`. This function is called in the `transformed parameters` block in order to obtain metabolite concentrations given parameter values. In the `model` block the values of the variable `metabolite_concentration` are used in the measurement model.




In [5]:
inference_model_file = 'stan_code/inference_model_linear.stan'

with open('stan_code/inference_model_linear.stan', 'r') as f:
    print(f.read())

functions {
#include rate_equations.stan
#include allostery.stan
  real [] get_fluxes(real[] metabolites, real[] params, real[] known_reals){  
    real empty_array[0];
    real free_enzyme_ratio_r2 = get_free_enzyme_ratio_uniuni(metabolites[2],
                                                             metabolites[3],
                                                             known_reals[2]*params[6],
                                                             known_reals[2]*params[7],
                                                             params[8],
                                                             params[5]);
    return {
      uniuni(metabolites[1], metabolites[2], known_reals[1]*params[2], known_reals[1]*params[3], params[4], params[1]),
      uniuni(metabolites[2], metabolites[3], known_reals[2]*params[6], known_reals[2]*params[7], params[8], params[5])
        * get_regulatory_effect(empty_array, {metabolites[2]}, free_enzyme_ratio_r2, empty_array, {params[

### Data munging

The code in the following cell reads the data from the input file and organises it into pandas and python objects that can be used as input for Stan.

In [6]:
rn = reaction_network.load_from_toml('linear.toml')
display(rn.metabolites)
display(rn.unbalanced_metabolite_priors)
display(rn.kinetic_parameters)


,name,is_unbalanced,stan_code
0,M1_ex,True,1
1,M1,False,2
2,M2,False,3
3,M2_ex,True,4


,metabolite,loc,scale,true_value,experiment_label,metabolite_code,experiment_code
0,M1_ex,1,0.05,2.2,condition_1,1,1
1,M2_ex,0,0.05,1.3,condition_1,4,1
2,M1_ex,1,0.05,2.9,condition_2,1,2
3,M2_ex,0,0.05,0.7,condition_2,4,2


,parameter,loc,scale,type,true_value,metabolite,reaction,is_allosteric,label,stan_code
0,Keq,0.0,0.05,kinetic,1.0,NaN,r1,0,r1_Keq,1
1,Kcat1,0.0,0.60,kinetic,1.3,NaN,r1,0,r1_Kcat1,2
2,Kcat2,0.0,0.60,kinetic,1.0,NaN,r1,0,r1_Kcat2,3
3,Ka,0.0,0.60,kinetic,0.4,NaN,r1,0,r1_Ka,4
4,Keq,1.1,0.50,kinetic,4.0,NaN,r2,0,r2_Keq,5
5,Kcat1,0.0,0.60,kinetic,1.3,NaN,r2,0,r2_Kcat1,6
6,Kcat2,0.0,0.60,kinetic,1.0,NaN,r2,0,r2_Kcat2,7
7,Ka,0.0,0.60,kinetic,0.4,NaN,r2,0,r2_Ka,8
8,dissociation_constant_t,0.0,0.60,allosteric,2.0,M1,r2,1,r2_dissociation_constant_t_M1,9
9,transfer_constant,0.0,0.60,allosteric,1.1,NaN,r2,1,r2_transfer_constant,10


### Simulating some  measurements
A separate simulation model, shown below, is used to generate measurement values. It has the same `functions` block as the inference model, but does not do any MCMC as no uncertainty needs to be taken into account.

In [7]:
simulation_model_file = 'stan_code/simulation_model_linear.stan'

with open(simulation_model_file, 'r') as f:
    print(f.read())

functions {
#include rate_equations.stan
#include allostery.stan
  real [] get_fluxes(real[] metabolites, real[] params, real[] known_reals){  
    real empty_array[0];
    real free_enzyme_ratio_r2 = get_free_enzyme_ratio_uniuni(metabolites[2], metabolites[3], known_reals[2]*params[6], known_reals[2]*params[7], params[8], params[5]);
    return {
      uniuni(metabolites[1], metabolites[2], known_reals[1]*params[2], known_reals[1]*params[3], params[4], params[1]),
      uniuni(metabolites[2], metabolites[3], known_reals[2]*params[6], known_reals[2]*params[7], params[8], params[5]) * get_regulatory_effect(empty_array, {metabolites[2]}, free_enzyme_ratio_r2, empty_array, {params[9]}, params[10]),
      uniuni(metabolites[3], metabolites[4], known_reals[3]*params[12], known_reals[3]*params[13], params[14], params[11])
    };
  }
  real[] get_odes(real[] fluxes){
    return {  
      0,
      1*fluxes[1]-1*fluxes[2],
      1*fluxes[2]-1*fluxes[3],
      0
    };
  }
  real[] steady_state_

The next cell uses the `true_value` entries from the input data to simulate some measurement values.

In [8]:
STEADY_STATE_TIME = 200
ABS_TOL = 1e-12
REL_TOL = 1e-13
MAX_STEPS = int(1e9)
SIMULATION_CSV_PATH = 'output_data/simulation_output.csv'

metabolites = rn.metabolites
metabolite_names = rn.stoichiometry.columns
unbalanced_metabolites = rn.metabolites.query('is_unbalanced')
balanced_metabolites = rn.metabolites.query('~is_unbalanced')
reaction_names = rn.stoichiometry.index
concentration_unbalanced = (
    rn.unbalanced_metabolite_priors
    .set_index(['metabolite_code', 'experiment_code'])
    ['true_value']
    .unstack()
    .values
    .tolist()
)
simulation_input_data = {
    'N_balanced': len(balanced_metabolites),
    'N_unbalanced': len(unbalanced_metabolites),
    'N_kinetic_parameter': len(rn.kinetic_parameters),
    'N_reaction': len(reaction_names),
    'N_experiment': len(rn.experiments),
    'N_known_real': len(rn.known_reals),
    'N_flux_measurement': len(rn.flux_measurements),
    'N_concentration_measurement': len(rn.concentration_measurements),
    'pos_balanced': balanced_metabolites['stan_code'].values,
    'pos_unbalanced': unbalanced_metabolites['stan_code'].values,
    'ix_experiment_concentration_measurement': rn.concentration_measurements['experiment_code'].values,
    'ix_experiment_flux_measurement': rn.flux_measurements['experiment_code'].values,
    'ix_metabolite_concentration_measurement': rn.concentration_measurements['metabolite_code'].values,
    'ix_reaction_flux_measurement': rn.flux_measurements['reaction_code'].values,
    'flux_measurement_scale': rn.flux_measurements['scale'].values,
    'concentration_measurement_scale': rn.concentration_measurements['scale'].values,
    'kinetic_parameter': rn.kinetic_parameters['true_value'].values,
    'concentration_unbalanced': concentration_unbalanced,
    'known_reals': rn.known_reals.drop('stan_code', axis=1).values.tolist(),
    'initial_time': 0,
    'steady_time': STEADY_STATE_TIME,
    'rel_tol': REL_TOL,
    'abs_tol': ABS_TOL,
    'max_steps': MAX_STEPS
}

model = cmdstanpy.Model(stan_file=simulation_model_file)
model.compile(include_paths=['stan_code'], overwrite=True)
    
stanfit = model.sample(
    data=simulation_input_data,
    csv_basename=SIMULATION_CSV_PATH,
    adapt_engaged=False,
    warmup_iters=0,
    sampling_iters=1,
    chains=1,
    cores=1
)

simulated_concentration_measurements = (
    rn.concentration_measurements[['metabolite_label', 'experiment_label']]
    .rename(columns={'metabolite_label': 'metabolite', 'experiment_label': 'experiment'})
    .assign(
        simulated_measurement=stanfit.get_drawset(['simulated_concentration_measurement']).iloc[0].values
    )
    .round(2)
)
simulated_flux_measurements = (
    rn.flux_measurements[['flux_label', 'experiment_label']]
    .rename(columns={'flux_label': 'reaction', 'experiment_label': 'experiment'})
    .assign(
        simulated_measurement=stanfit.get_drawset(['simulated_flux_measurement']).iloc[0].values
    )
    .round(2)

)

display(simulated_concentration_measurements)
display(simulated_flux_measurements)

INFO:cmdstanpy:stan to c++ (stan_code/simulation_model_linear.hpp)
INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/stancon_2019_metabolic_network_notebook/stan_code/simulation_model_linear
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


,metabolite,experiment,simulated_measurement
0,M1,condition_1,0.78
1,M2,condition_1,1.78
3,M1,condition_2,0.47
4,M2,condition_2,1.05


,reaction,experiment,simulated_measurement
2,r3,condition_1,0.05
5,r3,condition_2,0.13


### Sampling given simulated measurements

The next cell uses cmdstanpy to sample from the model, once ignoring the data for prior predictive checks and once using values simulated above.


In [ ]:
# Initialise and compile model
inference_model = cmdstanpy.Model(stan_file=inference_model_file)
inference_model.compile(include_paths=['stan_code'])

# Define inputs for prior and posterior and save them as json files 
unbalanced_loc, unbalanced_scale = (
    rn.unbalanced_metabolite_priors
    .set_index(['metabolite_code', 'experiment_code'])
    [col]
    .unstack()
    .values
    .tolist()
    for col in ['loc', 'scale']
)

inference_input_prior = {
    'N_balanced': len(balanced_metabolites),
    'N_unbalanced': len(unbalanced_metabolites),
    'N_kinetic_parameter': len(rn.kinetic_parameters),
    'N_reaction': len(reaction_names),
    'N_experiment': len(rn.experiments),
    'N_known_real': len(rn.known_reals),
    'N_flux_measurement': len(rn.flux_measurements),
    'N_concentration_measurement': len(rn.concentration_measurements),
    'pos_balanced': balanced_metabolites['stan_code'].values,
    'pos_unbalanced': unbalanced_metabolites['stan_code'].values,
    'ix_experiment_concentration_measurement': rn.concentration_measurements['experiment_code'].values,
    'ix_experiment_flux_measurement': rn.flux_measurements['experiment_code'].values,
    'ix_metabolite_concentration_measurement': rn.concentration_measurements['metabolite_code'].values,
    'ix_reaction_flux_measurement': rn.flux_measurements['reaction_code'].values,
    
    # Simulated measurements used here
    'flux_measurement': simulated_flux_measurements['simulated_measurement'].values,
    'concentration_measurement': simulated_concentration_measurements['simulated_measurement'].values,
    
    'flux_measurement_scale': rn.flux_measurements['scale'].values,
    'concentration_measurement_scale': rn.concentration_measurements['scale'].values,
    'prior_location_kinetic_parameter': rn.kinetic_parameters['loc'].values,
    'prior_scale_kinetic_parameter': rn.kinetic_parameters['scale'].values,
    'prior_location_unbalanced': unbalanced_loc,
    'prior_scale_unbalanced': unbalanced_scale,
    'known_reals': rn.known_reals.drop('stan_code', axis=1).values.tolist(),
    'initial_time': 0,
    'steady_time': STEADY_STATE_TIME,
    'rel_tol': REL_TOL,
    'abs_tol': ABS_TOL,
    'max_steps': MAX_STEPS,
    'LIKELIHOOD': 0
}
inference_input_posterior = inference_input_prior.copy()
inference_input_posterior['LIKELIHOOD'] = 1

here = os.path.abspath('')
inference_input_file_prior = os.path.join(here, 'output_data/inference_input_data_prior.json')
inference_input_file_posterior = os.path.join(here, 'output_data/inference_input_data_posterior.json')
cmdstanpy.utils.jsondump(inference_input_file_prior, inference_input_prior)
cmdstanpy.utils.jsondump(inference_input_file_posterior, inference_input_posterior)

# Generate samples
inference_csv_basename_prior = os.path.join(here, 'output_data/inference_output_linear_prior.csv')
inference_csv_basename_posterior = os.path.join(here, 'output_data/inference_output_linear_posterior.csv')

stanfit_prior = inference_model.sample(
    data=inference_input_file_prior,
    csv_basename=inference_csv_basename_prior,
    warmup_iters=200,
    sampling_iters=100,
    chains=2,
    cores=2,
    save_warmup=True
)
stanfit_posterior = inference_model.sample(
    data=inference_input_file_posterior,
    csv_basename=inference_csv_basename_posterior,
    warmup_iters=200,
    sampling_iters=100,
    chains=2,
    cores=2,
    save_warmup=True
)

# Display summaries
display(stanfit_prior.summary())
display(stanfit_posterior.summary())

            
shared_arviz_kwargs = dict(
    posterior_predictive=[
        'simulated_flux_measurement', 
        'simulated_concentration_measurement'
    ],
    observed_data={
        'simulated_flux_measurement': input_data['flux_measurement'],
        'simulated_concentration_measurement': input_data['concentration_measurement']
    },
    coords={
        'reactions': reaction_names,
        'metabolites': metabolite_names,
        'experiments': [x['label'] for x in ed.experiments],
        'parameter_names':  (
        [f'{x}-{y}' for x, y in zip(ed.kinetic_parameters['reaction'], ed.kinetic_parameters['parameter'])]
        ),
        'flux_measurements': (
            [f'{x}-{y}' for x, y in zip(ed.flux_measurements['reaction_code'], ed.flux_measurements['experiment_code'])]
        ),
        'concentration_measurements': (
            [f'{x}-{y}' for x, y in zip(ed.concentration_measurements['metabolite_code'], ed.concentration_measurements['experiment_code'])]
        )
    },
    dims={
        'concentration': ['metabolites', 'experiments'],
        'flux': ['reactions', 'experiments'],
        'kinetic_parameter': ['parameter_names'],
        'simulated_concentration_measurement': ['concentration_measurements'],
        'simulated_flux_measurement': ['flux_measurements'],

    }
)
infd_prior = arviz.from_cmdstanpy(stanfit_prior, **shared_arviz_kwargs)
infd_posterior = arviz.from_cmdstanpy(stanfit_posterior, **shared_arviz_kwargs)

INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/stancon_2019_metabolic_network_notebook/stan_code/inference_model_linear
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2


In [ ]:
shared_arviz_kwargs = dict(
    posterior_predictive=[
        'simulated_flux_measurement', 
        'simulated_concentration_measurement'
    ],
    observed_data={
        'simulated_flux_measurement': inference_input_posterior['flux_measurement'],
        'simulated_concentration_measurement': inference_input_posterior['concentration_measurement']
    },
    coords={
        'reactions': reaction_names,
        'metabolites': metabolite_names,
        'experiments': [x['label'] for x in rn.experiments],
        'parameter_names':  (
        [f'{x}.{y}' for x, y in zip(rn.kinetic_parameters['reaction'], rn.kinetic_parameters['parameter'])]
        ),
        'flux_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.flux_measurements['flux_label'], rn.flux_measurements['experiment_label'])]
        ),
        'concentration_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.concentration_measurements['metabolite_label'], rn.concentration_measurements['experiment_label'])]
        )
    },
    dims={
        'concentration': ['metabolites', 'experiments'],
        'flux': ['reactions', 'experiments'],
        'kinetic_parameter': ['parameter_names'],
        'simulated_concentration_measurement': ['concentration_measurements'],
        'simulated_flux_measurement': ['flux_measurements'],

    }
)
infd_prior = arviz.from_cmdstanpy(stanfit_prior, **shared_arviz_kwargs)
infd_posterior = arviz.from_cmdstanpy(stanfit_posterior, **shared_arviz_kwargs)

### Model checking
From the summary and lack of warnings from the diagnostic function we can see that the chains have mixed pretty well and that there were no post-warmup divergent transitions. From the summary of the `metabolite_flux` variables shows that these were mostly very close to zero, indicating that the system reached steady state in most iterations.


### Analysis
In order to ascertain whether the simulated experimental apparatus is powerful enough for it to learn about the network's parameters, we can look at each parameter's marginal prior and posterior distribution. If the model has learned about the parameter then there should be more probability mass around the true value in the marginal posterior distribution than in the prior.

This cell uses the analysis package [arviz](https://arviz-devs.github.io/arviz/index.html) to plot some quantiles of each parameter's marginal prior and posterior distribution alongside the true value that was fed into the simulation.

For some parameters we can see that the marginal posterior concentrates more closely around the true value than the marginal prior distribution. For other parameters this is not the case. Depending on which parameters we care most about, this test could help determine whether or not to carry out the experiments.

In [ ]:
f, [ax] = arviz.plot_forest(
    [infd_prior, infd_posterior],
    combined=True,
    model_names=['prior', 'posterior'],
    var_names=['kinetic_parameter'],
    figsize=[15, 10],
    colors=['tab:orange', 'tab:blue']
)
t = ax.set_title('How did the marginal parameter distributions change from prior to posterior?', fontsize=15)
lines = [l for axis in f.axes for l in axis.get_lines()]
for l, true_value in zip(lines[::-2], rn.kinetic_parameters['true_value'].tolist()):
    y = l.get_ydata()[0] - 1.15
    p = ax.plot(true_value, y, '+', color='r')
ax.legend([p[0], lines[1], lines[0]], ['True value', 'Prior', 'Posterior'], frameon=False, loc='upper left')
ax.semilogx()
plt.show()

The next cell makes a similar plot for the observed metabolite concentrations. Here the convergence towards the true value from prior to posterior is much higher than for the marginal parameter distributions. 

In [ ]:
posterior_low, posterior_med, posterior_high = (
    infd_posterior.posterior_predictive['simulated_concentration_measurement'].quantile(q, dim=['chain', 'draw'])
    for q in [0.1, 0.5, 0.9]
)
prior_low, prior_med, prior_high = (
    infd_prior.posterior_predictive['simulated_concentration_measurement'].quantile(q, dim=['chain', 'draw'])
    for q in [0.1, 0.5, 0.9]
)
obs = infd_posterior.observed_data['simulated_concentration_measurement']
x = np.linspace(infd_posterior.posterior_predictive['simulated_concentration_measurement'].min(), 
                infd_posterior.posterior_predictive['simulated_concentration_measurement'].max(), 100)
labels = posterior_med.coords['concentration_measurements']

f, ax = plt.subplots(figsize=[15, 5])

ax.scatter(prior_med, obs, label='', color='tab:orange')
ax.hlines(obs, prior_low, prior_high, color='tab:orange', label='Prior')

ax.scatter(posterior_med, obs, label='', color='tab:blue')
ax.hlines(obs, posterior_low, posterior_high, color='tab:blue', label='Posterior', alpha=1)
for label, observed_value, prior_med_value in zip(labels.values, obs.values, prior_med.values):
    ax.text(prior_med_value, observed_value * 1.1, label)


ax.plot(x, x, color='r', linestyle='--', label='y=x')
ax.semilogx()
ax.semilogy()
legend = ax.legend(frameon=False)
text = ax.set(title='How did the metabolite concentration predictions change from prior to posterior?',
              xlabel='Modelled value',
              ylabel='Observed value')

Finally, the following plot shows the posterior correlation between each parameter. We can see that although there are some correlations, there are no troubling funnel or banana shapes.

In [ ]:
axes = arviz.plot_pair(infd_posterior, 
                       var_names=['kinetic_parameter'], 
                       coords={'parameter_names': ['r1.Kcat1', 'r1.Ka', 'r1.Kcat2', 'r2.Kcat1']},
                       figsize=[15, 10])


## Acknowledgements
The research was produced as part of the project '[Quantitative modelling of cell metabolism](https://www.biosustain.dtu.dk/research/scientific-sections/quantitative-modelling-of-cell-metabolism)', which is part of the DTU Biosustain department and is funded by the [Novo Nordisk Foundation](https://novonordiskfonden.dk/en/).


## References

Cleland, W. W. 1963. “The Kinetics of Enzyme-Catalyzed Reactions with Two or More Substrates or Products: I. Nomenclature and Rate Equations.” _Biochimica et Biophysica Acta (BBA) - Specialized Section on Enzymological Subjects 67_ (January): 104–37. doi:10.1016/0926-6569(63)90211-6.

Popova, S. V., and E. E. Sel’kov. 1975. “Generalization of the Model by Monod, Wyman and Changeux for the Case of a Reversible Monosubstrate Reaction.” _FEBS Letters_ 53 (3): 269–73. doi:10.1016/0014-5793(75)80034-2.

Saa, Pedro A., and Lars K. Nielsen. 2016. “Construction of Feasible and Accurate Kinetic Models of Metabolism: A Bayesian Approach.” _Scientific Reports_ 6 (1). doi:10.1038/srep29635.

———. 2017. “Formulation, Construction and Analysis of Kinetic Models of Metabolism: A Review of Modelling Frameworks.” _Biotechnology Advances_ 35 (8): 981–1003. doi:10.1016/j.biotechadv.2017.09.005.

Tsigkinopoulou, Areti, Syed Murtuza Baker, and Rainer Breitling. 2017. “Respectful Modeling: Addressing Uncertainty in Dynamic System Models for Molecular Biology.” _Trends in Biotechnology_ 35 (6): 518–29. doi:10.1016/j.tibtech.2016.12.008.
